In [7]:

# ============================================================
# Walmart Mini Dashboard (Regression + Accuracy Only)
# Run: python walmart_dashboard.py
# Open: http://127.0.0.1:8021
# ============================================================

import pandas as pd
import numpy as np
from threading import Timer
import webbrowser

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor

import plotly.graph_objects as go
import plotly.express as px

from dash import Dash, dcc, html, Input, Output


# ------------------------------------------------------------
# 1) Load + Feature Engineering
# ------------------------------------------------------------
df = pd.read_csv("Walmart.csv")

df["Date"] = pd.to_datetime(df["Date"], dayfirst=True, errors="coerce")
df = df.dropna(subset=["Date", "Weekly_Sales"]).copy()
df = df.sort_values(["Date", "Store"]).reset_index(drop=True)

df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Week"] = df["Date"].dt.isocalendar().week.astype(int)

# Classification label (median split)
df["High_Sales"] = (df["Weekly_Sales"] > df["Weekly_Sales"].median()).astype(int)

feature_cols = [
    "Store", "Holiday_Flag", "Temperature",
    "Fuel_Price", "CPI", "Unemployment",
    "Year", "Month", "Week"
]

X = df[feature_cols].copy()
y = df["Weekly_Sales"].copy()
y_class = df["High_Sales"].copy()


# ------------------------------------------------------------
# 2) Train/Test Split (Time-based)
# ------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, shuffle=False
)
test_idx = X_test.index
y_test_class = y_class.iloc[test_idx]


# ------------------------------------------------------------
# 3) Train Regression Model
# ------------------------------------------------------------
model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    objective="reg:squarederror",
    tree_method="hist"
)
model.fit(X_train, y_train)

pred = model.predict(X_test)

# Regression global metrics
rmse_global = mean_squared_error(y_test, pred) ** 0.5
mae_global = mean_absolute_error(y_test, pred)


# ------------------------------------------------------------
# 4) Convert Regression → Binary Classification (Accuracy Only)
# ------------------------------------------------------------
threshold = np.median(pred)
pred_class = (pred > threshold).astype(int)

# Global accuracy
global_accuracy = accuracy_score(y_test_class, pred_class)


# ------------------------------------------------------------
# 5) Build Result Table
# ------------------------------------------------------------
res = df.loc[test_idx, ["Date", "Store", "Holiday_Flag", "Weekly_Sales"]].copy()

res["Predicted"] = pred
res["Residual"] = res["Weekly_Sales"] - res["Predicted"]
res["Abs_Error"] = res["Residual"].abs()

# Add classification columns
res["True_Class"] = y_test_class.values
res["Pred_Class"] = pred_class

res = res.sort_values(["Store", "Date"]).reset_index(drop=True)
stores = sorted(res["Store"].unique())


# ------------------------------------------------------------
# 6) Store-wise ACCURACY ONLY
# ------------------------------------------------------------
def store_accuracy(store_id):
    sd = res[res["Store"] == store_id]
    if len(sd) == 0:
        return 0.0

    return accuracy_score(sd["True_Class"], sd["Pred_Class"])


# ------------------------------------------------------------
# 7) Store Regression Metrics
# ------------------------------------------------------------
def store_kpis(store_id: int):
    sd = res[res["Store"] == store_id]

    if len(sd) == 0:
        return 0, 0, 0, 0, 0

    rmse_s = mean_squared_error(sd["Weekly_Sales"], sd["Predicted"]) ** 0.5
    mae_s = mean_absolute_error(sd["Weekly_Sales"], sd["Predicted"])
    avg_a = sd["Weekly_Sales"].mean()
    avg_p = sd["Predicted"].mean()
    n = len(sd)

    return rmse_s, mae_s, avg_a, avg_p, n


# ------------------------------------------------------------
# 8) KPI CARDS (Accuracy included)
# ------------------------------------------------------------
def kpi_cards(rmse_s, mae_s, avg_a, avg_p, n, accuracy_s):
    box_style = {
        "padding": "10px",
        "border": "1px solid #ddd",
        "borderRadius": "8px",
        "width": "20%",
        "textAlign": "center"
    }

    return [
        html.Div([html.H4("Rows"), html.H3(f"{n}")], style=box_style),
        html.Div([html.H4("Store RMSE"), html.H3(f"{rmse_s:,.0f}")], style=box_style),
        html.Div([html.H4("Store MAE"), html.H3(f"{mae_s:,.0f}")], style=box_style),
        html.Div([html.H4("Avg Actual"), html.H3(f"{avg_a:,.0f}")], style=box_style),
        html.Div([html.H4("Accuracy"), html.H3(f"{accuracy_s:.2f}")], style=box_style),
    ]


# ------------------------------------------------------------
# 9) CHART FUNCTIONS
# ------------------------------------------------------------
def fig_store_line(store_id):
    sd = res[res["Store"] == store_id].sort_values("Date")
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=sd["Date"], y=sd["Weekly_Sales"], mode="lines", name="Actual"))
    fig.add_trace(go.Scatter(x=sd["Date"], y=sd["Predicted"], mode="lines", name="Predicted", line=dict(dash="dash")))
    return fig


def fig_store_residual(store_id):
    sd = res[res["Store"] == store_id].sort_values("Date")
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=sd["Date"], y=sd["Residual"], mode="lines", name="Residual"))
    fig.add_hline(y=0)
    return fig


def fig_store_hist(store_id):
    sd = res[res["Store"] == store_id]
    return px.histogram(sd, x="Residual", nbins=30)


def fig_store_scatter(store_id):
    sd = res[res["Store"] == store_id]
    return px.scatter(sd, x="Weekly_Sales", y="Predicted", color="Holiday_Flag")


def fig_store_holiday_error(store_id):
    sd = res[res["Store"] == store_id]
    return px.box(sd, x="Holiday_Flag", y="Abs_Error")


# ------------------------------------------------------------
# 10) DASH APP
# ------------------------------------------------------------
app = Dash(__name__)
app.title = "Walmart Dashboard - Accuracy Only"

app.layout = html.Div([
    html.H2("Walmart Dashboard — Regression + Accuracy Only"),

    dcc.Dropdown(
        id="store_dd",
        options=[{"label": f"Store {s}", "value": s} for s in stores],
        value=stores[0],
        clearable=False,
        style={"width": "250px"}
    ),

    html.Div(id="kpi_row", style={"display": "flex", "gap": "10px", "marginTop": "20px"}),

    dcc.Graph(id="g_store_line"),

    html.Div([
        html.Div([dcc.Graph(id="g_store_residual")], style={"width": "50%"}),
        html.Div([dcc.Graph(id="g_store_hist")], style={"width": "50%"}),
    ], style={"display": "flex"}),

    html.Div([
        html.Div([dcc.Graph(id="g_store_scatter")], style={"width": "50%"}),
        html.Div([dcc.Graph(id="g_store_holiday_err")], style={"width": "50%"}),
    ], style={"display": "flex"}),
])


@app.callback(
    Output("kpi_row", "children"),
    Output("g_store_line", "figure"),
    Output("g_store_residual", "figure"),
    Output("g_store_hist", "figure"),
    Output("g_store_scatter", "figure"),
    Output("g_store_holiday_err", "figure"),
    Input("store_dd", "value")
)
def update_all(store_id):
    rmse_s, mae_s, avg_a, avg_p, n = store_kpis(store_id)
    accuracy_s = store_accuracy(store_id)

    return (
        kpi_cards(rmse_s, mae_s, avg_a, avg_p, n, accuracy_s),
        fig_store_line(store_id),
        fig_store_residual(store_id),
        fig_store_hist(store_id),
        fig_store_scatter(store_id),
        fig_store_holiday_error(store_id)
    )


# ------------------------------------------------------------
# 11) RUN SERVER
# ------------------------------------------------------------
def open_browser():
    webbrowser.open_new("http://127.0.0.1:8021")

if __name__ == "__main__":
    Timer(1, open_browser).start()
    app.run(host="127.0.0.1", port=8021, debug=True)
